# Imports

In [1]:
from gensim.models import KeyedVectors

In [2]:
pt_model = KeyedVectors.load_word2vec_format('wiki.pt/wiki.pt.vec')

In [3]:
words = []
for word in pt_model.vocab:
    words.append(word)

In [4]:
print("Number of Tokens: {}".format(len(words)))

Number of Tokens: 592108


In [5]:
print("Dimension of a word vector: {}".format(
    len(pt_model[words[0]])
))

Dimension of a word vector: 300


In [25]:
words[1000]

'artes'

In [26]:
print("Vector components of a word: {}".format(
    pt_model[words[0]]
))

Vector components of a word: [-0.04817     0.11343    -0.21973    -0.096098   -0.0053306   0.20583
  0.15409     0.20406    -0.010307   -0.045906   -0.019991   -0.14881
  0.071691   -0.010353   -0.040639   -0.19218    -0.054845   -0.099474
 -0.032876   -0.19517     0.091734   -0.1428      0.031234    0.069291
 -0.014771   -0.0059432  -0.092041   -0.11749    -0.012621   -0.037947
  0.045571   -0.086255    0.0032468   0.038197    0.05541    -0.14174
  0.046568   -0.24105    -0.020556    0.075958    0.046163    0.14249
  0.053788    0.073211   -0.020421    0.068428    0.078169    0.10405
  0.058263   -0.22894    -0.083738    0.027697    0.060742    0.12547
  0.018675   -0.059984    0.027855    0.12125    -0.089253    0.053595
 -0.076205   -0.11818     0.016257    0.031477   -0.080403    0.079504
 -0.24153    -0.069171    0.019415   -0.040792    0.081118   -0.072191
 -0.026516   -0.17692    -0.033728   -0.1433     -0.0073834  -0.043411
 -0.16559     0.084007   -0.043258    0.075611   -0.03

In [15]:
find_similar_to = 'carro'

for similar_word in pt_model.similar_by_word(find_similar_to):
    print("Word: {0}, Similarity: {1:.2f}".format(
        similar_word[0], similar_word[1]
    ))

Word: carros, Similarity: 0.75
Word: caminhão, Similarity: 0.73
Word: automóvel, Similarity: 0.70
Word: carroção, Similarity: 0.69
Word: motorista, Similarity: 0.68
Word: caminhonete, Similarity: 0.67
Word: capotar, Similarity: 0.66
Word: furgão, Similarity: 0.65
Word: carroçeria, Similarity: 0.65
Word: capotando, Similarity: 0.65


/home/luke/anaconda3/envs/tf-gpu/lib/python3.6/site-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


In [13]:
word_add = ['dormir', 'beber']
word_sub = ['cama']


for resultant_word in pt_model.most_similar(
    positive=word_add, negative=word_sub
):
    print("Word : {0} , Similarity: {1:.2f}".format(
        resultant_word[0], resultant_word[1]
    ))

Word : comer , Similarity: 0.61
Word : beberem , Similarity: 0.60
Word : beberdes , Similarity: 0.59
Word : beberes , Similarity: 0.57
Word : bebemorar , Similarity: 0.57
Word : beberagem , Similarity: 0.56
Word : bebem , Similarity: 0.56
Word : bebera , Similarity: 0.56
Word : dormirem , Similarity: 0.55
Word : beberá , Similarity: 0.55


/home/luke/anaconda3/envs/tf-gpu/lib/python3.6/site-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


In [ ]:
def gkern(kernlen=5, nsig=3):
    """Returns a 2D Gaussian kernel array."""

    interval = (2*nsig+1.)/(kernlen)
    x = np.linspace(-nsig-interval/2., nsig+interval/2., kernlen+1)
    kern1d = np.diff(st.norm.cdf(x))
    kernel_raw = np.sqrt(np.outer(kern1d, kern1d))
    kernel = kernel_raw/kernel_raw.sum()
    return kernel

In [3]:
from scipy.signal import convolve2d


class Classifier:
    
    def w2v(self, words):
        pass
    
    def calc_var(self, words):
            vs = np.array([self.model[word] for word in words])
            mean = np.mean(vs, axis=0)
            
            return np.sum((vs - mean)**2)/vs.shape[0]
            
            
    def calc_dists(self, word, kws):
        dists = []
        for kw in kws:
            dists += [self.model.similarity(word, kw)]
    
        return np.array(dists)
    
    
    def classify(self, url, kws, labels, dist_thresh=0.5, kws_thresh=0.5):
        words = get_wd_list(url)
        dists = []
        for word in words:
            dists += [self.calc_dists(word, kws)]
            
        dists = np.array(dists)
        
        dists = convolve2d(dists, gkern(), mode='same')
        
        df = pd.DataFrame(dists, columns=kws)
        
        result = dict()
        for label in labels:
            result[label.name] = (df[label.keywords].mean(axis=0) > dist_thresh).sum() > kws_thresh
            
        return result        
        